<img class='pull-right' src='Images/cgs.png'>

# CGS webinar - CPT processing tutorial

This notebook presents the workflow for CPT processing with ```groundhog```, presented during a webinar of the Canadian Geotechnical Society eduction committee on the Future of Geotechnical Engineering Education.

This notebook shows how ```groundhog``` functionality can be used to easily load, process and present CPT data. The abilities of Python and ```groundhog``` to automate processing and present results from several CPTs are also illustrated.

The CPT information used in this tutorial is from the [Borssele offshore windfarm site](https://offshorewind.rvo.nl/studiesborsseleI) which is used under a Creative Commons 4.0 license.

## Loading libraries

For the processing of CPT data, we will need the ```PCPTProcessing``` class from ```groundhog```. This class encodes all required functionality to perform CPT processing. We can import it as follows:

In [1]:
from groundhog.siteinvestigation.insitutests.pcpt_processing import PCPTProcessing

We will also import the Pandas library for data processing.

In [2]:
import pandas as pd

## Data loading

We will process CPT data for test location WFS1-2 at the Borssele offshore wind farm site. The data for this CPT is provided in AGS 4.0 format, a common data transfer format in geotechnical engineering.

```groundhog``` allows import of CPT data from many different sources. We will start be creating a ```PCPTProcessing``` object for the selected PCPT.

In [3]:
cpt = PCPTProcessing(title="WFS1-2")

Loading the data can happen using the ```load_ags``` method. Internally, this makes use of the AGS conversion functionality in ```groundhog``` which generates a dataframe from an AGS file.

When looking at the AGS file, you can see that the headers are not very straightforward. When loading CPT data, ```groundhog``` expects the following columns:

   - Depth in m - Default column key = ```z [m]```
   - Cone tip resistance in MPa - Default column key = ```qc [MPa]```
   - Sleeve friction in MPa (optional) - Default column key = ```fs [MPa]```
   - Pore pressure at the shoulder in MPa (optional) - Default column key = ```u2 [MPa]```
   
For CPTs with multiple pushes, we also need to tell ```groundhog``` which column contains the identifier for the push. Here, there is only one push.
   
The AGS files has different names for these required keys, so we need to tell ```groundhog``` what they are. Moreover, $ f_s $ and $ u_2 $ have units of kPa so we need to convert to MPa by specifying the appropriate multiplier.

In [6]:
cpt.load_ags(
    path="Data/N6016_BH_WFS1-2_AGS4_150909.ags",
    z_key="SCPT_DPTH [m]", qc_key="SCPT_RES [MN/m2]", fs_key="SCPT_FRES [kN/m2]", u2_key="SCPT_PWP2 [kN/m2]",
    fs_multiplier=0.001, u2_multiplier=0.001,
    push_key="SCPG_TESN")

Plotting the raw CPT data is then simply achieved using the ```plot_raw_pcpt``` methods. Note that plotting ranges and tick mark intervals can be customised (as shown here for the pore pressure).

In [7]:
cpt.plot_raw_pcpt(u2_range=(-0.5, 2), u2_tick=0.25)

The CPT data itself is contained in the ```.data``` attribute, which we can print to the notebook (```.head()``` only plots the first five rows).

In [8]:
cpt.data.head()

,LOCA_ID,Push,z [m],qc [MPa],fs [MPa],u2 [MPa],SCPT_FRR [%],SCPT_QT [MN/m2],SCPT_QNET [MN/m2],SCPT_BQ,FILE_FSET
0,CPT_WFS1_2,1,0.00,0.003,NaN,NaN,NaN,0.004,0.004,0.4399,NaN
1,CPT_WFS1_2,1,0.02,0.014,NaN,0.0022,NaN,0.015,0.014,0.1307,NaN
2,CPT_WFS1_2,1,0.04,0.029,NaN,0.0020,NaN,0.030,0.029,0.0656,NaN
3,CPT_WFS1_2,1,0.06,0.051,NaN,0.0019,NaN,0.052,0.050,0.0277,NaN
4,CPT_WFS1_2,1,0.08,0.072,0.001389,0.0019,2.102,0.073,0.071,0.0167,NaN


## Data correction and normalisation

Since the CPT data has a pore pressure sensor at the shoulder, we need to correct for the unequal area effect. Since the pore water is also pushing on the rear of the cone, the total cone resistance $ q_t $ is less than the measured cone tip resistance $ q_c $.

### Cone properties

To apply this correction, we need to know the cone properties. ```groundhog``` expects the cone properties in a standard format, which we can load:

In [9]:
from groundhog.siteinvestigation.insitutests.pcpt_processing import DEFAULT_CONE_PROPERTIES
DEFAULT_CONE_PROPERTIES

,Depth from [m],Depth to [m],area ratio [-],Cone type,Cone base area [cm2],Cone sleeve_area [cm2],Sleeve cross-sectional area top [cm2],Sleeve cross-sectional area bottom [cm2]
0,0,20,0.8,U,10,150,NaN,NaN


This table contains the required cone properties. We can either create a similar table in Excel and load it, or replace the values in the dataframe.

The AGS file also contains the cone properties, so we can load them.

In [10]:
from groundhog.general.agsconversion import AGSConverter
converter = AGSConverter(path="Data/N6016_BH_WFS1-2_AGS4_150909.ags")
converter.create_dataframes(selectedgroups=['SCPG'], verbose_keys=True, use_shorthands=True)
converter.data['SCPG'].head()

,Location identifier,Test reference or push number,Cone test type,Cone reference,Ac [cm2],Nominal rate of penetration of the cone [mm/s],Type of filter material used,Friction reducer used,Groundwater level [m],Origin of water level in SCPG_WAT,...,Details of weather and environmental conditions during test,Subcontractors name,Standard followed for testing,Accrediting body and reference number (when appropriate),Area ratio,Sleeve area ratio,Associated file reference (eg cone calibration records),Cone diameter [mm],Sleeve length [mm],Sleeve area [cm2]
0,CPT_WFS1_2,1,PC,CP15-CF75PB20SN2 1701-2483,15.0,20.0,NaN,N,NaN,NaN,...,NaN,NaN,NEN 5140,NaN,0.58,0.01392,NaN,43.85,143.6,198.9


In [11]:
from groundhog.general.soilprofile import read_excel

The cone properties can either be converted from the dataframe above, or they can be read from an Excel file. Note that the cone properties need to be loaded as a ```groundhog``` ```SoilProfile``` object to make use of the functionality embedded in the object.

In [12]:
cone_props = read_excel("Data/WFS1-2_cone_props.xlsx")
cone_props

,Depth from [m],Depth to [m],area ratio [-],Cone type,Cone base area [cm2],Cone sleeve_area [cm2],Sleeve cross-sectional area top [cm2],Sleeve cross-sectional area bottom [cm2]
0,0,30,0.58,PC,15,198.9,NaN,NaN


### Layering

We also need to specify a (preliminary) layering with the associated total unit weights and soil types. We can encode this programmatically or load from Excel. 

In [13]:
layering = read_excel("Data/WFS1-2_layering.xlsx")
layering

,Depth from [m],Depth to [m],Total unit weight [kN/m3],Soil type
0,0.00,5.28,19.0,SAND
1,5.28,6.50,17.0,CLAY
2,6.50,11.80,19.5,SAND
3,11.80,18.70,18.0,SAND/CLAY
4,18.70,22.70,20.0,SAND
5,22.70,30.00,17.0,CLAY


### Mapping cone properties and layering

Once the layering and cone properties are known, we can map them to the cone data. This means that each data point (i.e. each depth) will get a cone properties and a layer assigned to it. This will allow us to do further calculations.

In addition to the mapping, a vertical stress calculation is also performed. By default, the water level is assumed at the surface, but we can put it at another level to represent a soil profile where the phreatic surface is some distance below the ground. Since this is an offshore CPT, the water level is indeed at 0m.

In [14]:
cpt.map_properties(layer_profile=layering, cone_profile=cone_props, waterlevel=0)

In [15]:
cpt.data.head()

,LOCA_ID,Push,z [m],qc [MPa],fs [MPa],u2 [MPa],SCPT_FRR [%],SCPT_QT [MN/m2],SCPT_QNET [MN/m2],SCPT_BQ,...,Total unit weight [kN/m3],Cone type,area ratio [-],Cone base area [cm2],Cone sleeve_area [cm2],Sleeve cross-sectional area top [cm2],Sleeve cross-sectional area bottom [cm2],Vertical total stress [kPa],Water pressure [kPa],Vertical effective stress [kPa]
0,CPT_WFS1_2,1,0.00,0.003,NaN,NaN,NaN,0.004,0.004,0.4399,...,19.0,PC,0.58,15.0,198.9,NaN,NaN,0.00,0.000,0.000
1,CPT_WFS1_2,1,0.02,0.014,NaN,0.0022,NaN,0.015,0.014,0.1307,...,19.0,PC,0.58,15.0,198.9,NaN,NaN,0.38,0.205,0.175
2,CPT_WFS1_2,1,0.04,0.029,NaN,0.0020,NaN,0.030,0.029,0.0656,...,19.0,PC,0.58,15.0,198.9,NaN,NaN,0.76,0.410,0.350
3,CPT_WFS1_2,1,0.06,0.051,NaN,0.0019,NaN,0.052,0.050,0.0277,...,19.0,PC,0.58,15.0,198.9,NaN,NaN,1.14,0.615,0.525
4,CPT_WFS1_2,1,0.08,0.072,0.001389,0.0019,2.102,0.073,0.071,0.0167,...,19.0,PC,0.58,15.0,198.9,NaN,NaN,1.52,0.820,0.700


### Normalisation

With the cone properties and stresses being known for each depth, the normalisation formulae can be applied. These formulae are encoded in ```groundhog``` and explained in the documentation. The method ```normalise_pcpt``` applies them all at once.

Columns with the normalised properties are added to the ```.data``` attribute of our ```PCPTProcessing``` object.

In [39]:
cpt.normalise_pcpt()
cpt.data['qc [MPa]'].rolling(20).mean()

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15          NaN
16          NaN
17          NaN
18          NaN
19      0.40785
20      0.44665
21      0.48450
22      0.52035
23      0.55455
24      0.58815
25      0.62105
26      0.65145
27      0.67910
28      0.70215
29      0.72140
         ...   
1471    5.45210
1472    5.39760
1473    5.33945
1474    5.28065
1475    5.22625
1476    5.18665
1477    5.15935
1478    5.14535
1479    5.13890
1480    5.13295
1481    5.12675
1482    5.12250
1483    5.12170
1484    5.12625
1485    5.14225
1486    5.16510
1487    5.19340
1488    5.22745
1489    5.26745
1490    5.31345
1491    5.36625
1492    5.41795
1493    5.46590
1494    5.50895
1495    5.54180
1496    5.55780
1497    5.55800
1498    5.54095
1499    5.51490
1500    5.48780
Name: qc [MPa], Length: 

We can also plot the normalised properties and check the validity of our layering selection.

In [17]:
cpt.plot_normalised_pcpt()

### Data export

The processing we have just done can be saved in an Excel file which contains the CPT data, selected layering, cone properties and the location of the CPT.

In [18]:
cpt.to_excel(output_path="Data/Excel export.xlsx")

## Soil type classification

### Robertson charts

The Robertson chart allows us to classify soil types based on the combination of normalised cone tip resistance and normalised friction ratio or pore pressure ratio.

```groundhog``` does this automatically and plots the combinations for the selected layers into a Robertson chart. This also shows that this data is often scattered and judgement needs to be applied.

```groundhog``` needs to know where the images for the Robertson charts are located. Here the files ```robertsonFr.png``` and ```robertsonBq.png``` are located in the subdirectory ```Images```. We can tell ```groundhog``` to look for the files in that location.

In [19]:
cpt.plot_robertson_chart(backgroundimagedir="Images")

## Soil parameter correlations

Correlation between CPT properties and other soil mechanics parameters can be applied to obtain parameters for geotechnical calculation. 

A common example is to derive the relative density for sand and the undrained shear strength for cohesive soils.

```groundhog``` implements a number of soil parameter correlations and provides a standard interface for applying them.

Each soil parameter correlation is implemented as a Python function in the module ```groundhog.siteinvestigation.insitutests.pcpt_correlations```. Each function is fully documented and validation is applied. We can check which correlations are available:

In [20]:
from groundhog.siteinvestigation.insitutests.pcpt_processing import CORRELATIONS
CORRELATIONS

{'Ic Robertson and Wride (1998)': <function groundhog.siteinvestigation.insitutests.pcpt_correlations.behaviourindex_pcpt_robertsonwride(qt, fs, sigma_vo, sigma_vo_eff, atmospheric_pressure=100.0, ic_min=1.0, ic_max=4.0, zhang_multiplier_1=0.381, zhang_multiplier_2=0.05, zhang_subtraction=0.15, robertsonwride_coefficient1=3.47, robertsonwride_coefficient2=1.22, **kwargs)>,
 'Isbt Robertson (2010)': <function groundhog.siteinvestigation.insitutests.pcpt_correlations.behaviourindex_pcpt_nonnormalised(qc, Rf, atmospheric_pressure=100.0, **kwargs)>,
 'Gmax Rix and Stokoe (1991)': <function groundhog.siteinvestigation.insitutests.pcpt_correlations.gmax_sand_rixstokoe(qc, sigma_vo_eff, multiplier=1634.0, qc_exponent=0.25, stress_exponent=0.375, **kwargs)>,
 'Gmax Mayne and Rix (1993)': <function groundhog.siteinvestigation.insitutests.pcpt_correlations.gmax_clay_maynerix(qc, multiplier=2.78, exponent=1.335, **kwargs)>,
 'Gmax Puechen (2020)': <function groundhog.siteinvestigation.insitutests

Here, we will apply the following correlations:

   - ```'Dr Jamiolkowski et al (2003)'``` for relative density in cohesionless soils
   - ```'Su Rad and Lunne (1988)'``` for undrained shear strength of cohesive soils
   
When looking at our layering, we can see that we have a interbedded ```SAND/CLAY``` layer for which we need to determine whether the behaviour will be cohesionless or cohesive. Due to the excess pore pressures measured, we will treat the layer as cohesive.

### Relative density

The relative density is calculated with the following formulae (see docs):

$$ D_{r,dry} = \frac{1}{2.96} \cdot \ln \left[ \frac{q_c / P_a}{24.94 \cdot \left( \frac{\sigma_{m}^{\prime}}{P_a} \right)^{0.46} } \right]\\D_{r,sat} = \left(  \frac{-1.87 + 2.32 \cdot \ln \left[ \frac{q_c}{\sqrt{P_a + \sigma_{vo}^{\prime}}} \right] }{100} \right) \cdot \frac{D_{r,dry}}{100} $$

The function requires cone tip resistance $ q_c $, vertical effective stress $ \sigma_{vo}^{\prime} $ and also the coefficient of lateral earth pressure $ K_0 $ to calculate the mean effective stress $ \sigma_m^{\prime} $

The function returns a dictionary with the keys ```Dr dry [-]``` for relative density in dry soil and ```Dr sat [-]``` for relative density in saturated soil.

We can apply the correlation with the ```apply_correlation``` method and then need to enter the following information:

   - The title of the correlation to be used (```'Dr Jamiolkowski et al (2003)'```)
   - The column name in which the result will be stored (```'Dr [-]'```)
   - The dictionary key from the function output which will be used (```'Dr sat [-]'```)
   - The coefficient of lateral earth pressure (```k0```) which is an argument for the correlation
   - Which soil types to apply the correlation to. Here, we will only apply it to ```SAND```. Note that we need to supply a Python list here.
   
Note that ```groundhog``` will issue warnings when we go outside the validation ranges of the function, but the calculation will not break down.

In [26]:
cpt.apply_correlation('Dr Jamiolkowski et al (2003)', outkey='Dr [-]', resultkey='Dr sat [-]',
                      k0=0.8, sigma_vo_eff__min=10,
                      apply_for_soiltypes=['SAND',])

We can check the numerical output:

In [27]:
cpt.data.head()

,LOCA_ID,Push,z [m],qc [MPa],fs [MPa],u2 [MPa],SCPT_FRR [%],SCPT_QT [MN/m2],SCPT_QNET [MN/m2],SCPT_BQ,...,Vertical effective stress [kPa],qt [MPa],Delta u2 [MPa],Rf [%],Bq [-],Qt [-],Fr [%],qnet [MPa],ft [MPa],Dr [-]
0,CPT_WFS1_2,1,0.00,0.003,NaN,NaN,NaN,0.004,0.004,0.4399,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CPT_WFS1_2,1,0.02,0.014,NaN,0.0022,NaN,0.015,0.014,0.1307,...,0.175,0.014924,0.001995,NaN,0.137170,83.108571,NaN,0.014544,NaN,NaN
2,CPT_WFS1_2,1,0.04,0.029,NaN,0.0020,NaN,0.030,0.029,0.0656,...,0.350,0.029840,0.001590,NaN,0.054677,83.085714,NaN,0.029080,NaN,NaN
3,CPT_WFS1_2,1,0.06,0.051,NaN,0.0019,NaN,0.052,0.050,0.0277,...,0.525,0.051798,0.001285,NaN,0.025366,96.491429,NaN,0.050658,NaN,NaN
4,CPT_WFS1_2,1,0.08,0.072,0.001389,0.0019,2.102,0.073,0.071,0.0167,...,0.700,0.072798,0.001080,1.908019,0.015152,101.825714,1.948708,0.071278,0.001389,NaN


### Undrained shear strength

The undrained shear strength can be derived from the net cone resistance ($ q_{net} = q_t - \sigma_{vo} $) as follows:

$$ S_u = \frac{q_{net}}{N_k} $$

The factor $ N_k $ is usually determined based on correlations between the CPT and laboratory measurements. For North Sea soils, this value is typically between 15 and 20. We will apply an average factor of 17.5 here.

The correlation is applied using the same syntax as for the relative density, but here we will apply it to ```CLAY``` and ```SAND/CLAY```.

In [28]:
cpt.apply_correlation('Su Rad and Lunne (1988)', outkey='Su [kPa]', resultkey='Su [kPa]',
                      Nk=17.5,
                      apply_for_soiltypes=['CLAY', 'SAND/CLAY'])

### Visualising soil parameter correlations

```groundhog``` contains functionality for visualising CPT data using a relatively simple syntax. It is always instructive to show a color-coded soil profile next to soil parameter plots and that's what the method ```plot_properties_withlog``` does.

We first need to defined a Python dictionary with the colors for each soil type:

In [29]:
fillcolors = {
    "SAND": "yellow",
    "CLAY": "brown",
    "SAND/CLAY": "orange"
}

Then we need to tell the method which parameters to plot. The argument ```prop_keys``` takes a list of soil parameter title lists. In the example, we will plot three panels, one for $ q_c $, one for $ D_r $ and one for $ S_u $. Each panel only contains one trace, so we have lists of one element.

We can specify whether to show the legend or not (```showlegends``` argument), what the ranges for our X-axes are (```plot_ranges``` argument) and the tick intervals on our X-axes (```plot_ticks``` argument). The X-axis titles can be specified (```axis_titles``` argument, note the $ \LaTeX $ syntax).

Finally, we set the z-range of our plot (```zrange``` argument) and can customise the layout using Plotly syntax (```layout``` argument). The colors for the soil profile are passed in the ```fillcolordict``` argument.

In [30]:
logfig = cpt.plot_properties_withlog(
    prop_keys=[('qc [MPa]',), ('Dr [-]',), ('Su [kPa]',)],
    showlegends=((False,), (False,), (False,)),
    plot_ranges=((0, 100), (0, 1.2), (0, 500)),
    plot_ticks=(10, 0.2, 100),
    axis_titles=(r'$ q_c \ \text{[MPa]} $', r'$ D_r \ \text{[-]} $', r'$ S_u \ \text{[kPa]} $'),
    zrange=(30, 0),
    layout=dict(width=1000),
    fillcolordict=fillcolors
    )

## Visualising spatial variability

```groundhog``` allows the rapid creation of profile lines which plot multiple CPT traces along a profile. For this purpose, we need to set the position of the CPTs using the ```set_position``` method.

We will load 4 CPTs from their AGS files and plot the CPT traces on one profile.

This example also demonstrates the automation possibilities of Python and ```groundhog```. We can first create a list of the locations for the profile.

In [31]:
locations = ['WFS1-3', 'WFS1-2A', 'WFS1-5A', 'WFS1-6']

We will create an empty list to store the CPTs:

In [32]:
profile_cpts = []

We can then loop over the locations and create ```PCPTProcessing``` objects for each. The path is made parametric to allow reading the different files.

Once the data is read, the ```PCPTProcessing``` object is appended to the list ```profile_cpts```.

In [33]:
for _loc in locations:
    _profile_cpt = PCPTProcessing(title=_loc)
    _profile_cpt.load_ags(
        path="Data/N6016_BH_%s_AGS4_150909.ags" % _loc,
        z_key="SCPT_DPTH [m]", qc_key="SCPT_RES [MN/m2]", fs_key="SCPT_FRES [kN/m2]", u2_key="SCPT_PWP2 [kN/m2]",
        push_key="SCPG_TESN",
        fs_multiplier=0.001, u2_multiplier=0.001)
    profile_cpts.append(_profile_cpt)

We can of course check the output for an individual CPT by using Python's syntax for accessing list elements.

In [34]:
profile_cpts[0].plot_raw_pcpt()

We then need to fix the locations of our CPTs in space. Coordinates and elevations can be retrieved from survey reports. It is important to add the correct coordinate system. Note that the longitudinale profiles will not work with latitude and longitude information.

In [35]:
eastings = [499081.02, 502763.64, 505926.68, 508454.6]
northings = [5732354.88, 5732537.58, 5730101.12, 5728133.69]
elevations = [-32.8, -24.6, -25.5, -31.6]
SRID = 25831

We can loop over each CPT and set its position. The ```enumerate``` operator is very handy in this respect.

In [36]:
for i, _cpt in enumerate(profile_cpts):
    _cpt.set_position(easting=eastings[i], northing=northings[i], elevation=elevations[i], srid=SRID)

Plotting of the longitudinal profile is done by the ```plot_longitudinal_profile``` method. This method returns a Plotly figure and visualizes the CPT information along the profile.

In [37]:
from groundhog.siteinvestigation.insitutests.pcpt_processing import plot_longitudinal_profile

We need to determine the start and the end point of the profile and the band in which the algorithm will search for locations. The offset from the profile line is rendered in the plot. The CPT trace can be made bigger or smaller using the ```scale_factor``` argument.

In [38]:
pcpt_fig = plot_longitudinal_profile(
    cpts=profile_cpts,
    start='WFS1-3', end='WFS1-6',
    band=2000, scale_factor=10
    )

<img src="Images/groundhog_banner_wide.png">